In [ ]:
!pip install nltk

In [ ]:
! pip install spacy

In [ ]:
! pip install markovify

     |████████████████████████████████| 235 kB 6.2 MB/s 
  Created wheel for markovify: filename=markovify-0.9.3-py3-none-any.whl size=18622 sha256=a7d40a75635fc2916923d79ba3508c719df6d4576f8bec7a6cbe121c5b4250b8
  Stored in directory: /root/.cache/pip/wheels/d9/f0/5b/748a27bdf2496bd4df51acb9442dae516efce507ff4849813e
Successfully built markovify


In [ ]:
! pip install -m spacy download en


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -m


In [ ]:
import spacy
import re
import markovify
import nltk
from nltk.corpus import gutenberg
import warnings
warnings.filterwarnings('ignore')

nltk.download('gutenberg')
! python -m spacy download en


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
     |████████████████████████████████| 12.0 MB 5.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
# Inspect Gutenberg corpus

print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [ ]:
# Import novels as text objects

hamlet = gutenberg.raw('shakespeare-hamlet.txt')
macbeth = gutenberg.raw('shakespeare-macbeth.txt')
caesar = gutenberg.raw('shakespeare-caesar.txt')

# Print first 100 characters of each

print('\n Hamlet \n', hamlet[:200])
print('\n Macbeth \n', macbeth[:200])
print('\n Caesar \n', caesar[:200])


 Hamlet 
 [The Tragedie of Hamlet by William Shakespeare 1599]


Actus Primus. Scoena Prima.

Enter Barnardo and Francisco two Centinels.

  Barnardo. Who's there?
  Fran. Nay answer me: Stand & vnfold
your sel

 Macbeth 
 [The Tragedie of Macbeth by William Shakespeare 1603]


Actus Primus. Scoena Prima.

Thunder and Lightning. Enter three Witches.

  1. When shall we three meet againe?
In Thunder, Lightning, or in Rai

 Caesar 
 [The Tragedie of Julius Caesar by William Shakespeare 1599]


Actus Primus. Scoena Prima.

Enter Flauius, Murellus, and certaine Commoners ouer the Stage.

  Flauius. Hence: home you idle Creatures, g


In [1]:
# print(hamlet)

In [ ]:
# Text cleaning to remove unneeded spaces and indentations, punctuations

def text_cleaner(text):
  text = re.sub(r'--', ' ', text)
  text = re.sub('[\[].*?[\]]', '', text)
  text = re.sub(r'(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b','', text)
  text = ' '.join(text.split())
  return text

In [ ]:
# Remove chapter indicator

hamlet = re.sub(r'Chapter \d+', '', hamlet)
macbeth = re.sub(r'Chapter \d+', '', macbeth)
caesar = re.sub(r'Chapter \d+', '', caesar)

In [ ]:
# Apply cleaning function to corpus

hamlet = text_cleaner(hamlet)
caesar = text_cleaner(caesar)
macbeth = text_cleaner(macbeth)

In [3]:
# Parse cleaned novels

nlp = spacy.load('en')

hamlet_doc = nlp(hamlet)
macbeth_doc = nlp(macbeth)
caesar_doc = nlp(caesar)

hamlet_doc

In [2]:
hamlet_sents = ' '.join([sent.text for sent in hamlet_doc.sents if len(sent.text) > 1])
macbeth_sents = ' '.join([sent.text for sent in macbeth_doc.sents if len(sent.text) > 1])
caesar_sents = ' '.join([sent.text for sent in caesar_doc.sents if len(sent.text) > 1])

shakespeare_sents = hamlet_sents + macbeth_sents + caesar_sents

# inspect our text
print(shakespeare_sents)

In [ ]:
# Create text generator using markovify

generator_1 =  markovify.Text(shakespeare_sents, state_size = 3)

In [ ]:
# We will randomly generate 3 sentences

for i in range(3):
  print(generator_1.make_sentence())

None
None
My Honourable Lord, I will not come to day, tell them so Caes The cause is in my Will , I will not Macb.


In [ ]:
# We will randomly generate three more sentences of no more than 100 characters

for i in range(3):
  print(generator_1.make_short_sentence(max_chars=100))

With what, in the name of Honor Bru.
Your flashes of Merriment that were wont to take delight in the Tragedians of the City Ham.
Go too: you are not fit Ham.


In [ ]:
# Next we will use spacy's part of speech to generate more legible text

class POSifiedText(markovify.Text):

  def word_split(self, sentence):
    return ['::'.join((word.orth_, word.pos_)) for word in nlp(sentence)]

  def word_join(self, words):
    sentence = ' '.join(word.split('::')[0] for word in words)
    return sentence

# Call the class on our text

generator_2 = POSifiedText(shakespeare_sents, state_size = 3)

In [ ]:
# Now we will use the above generator to generate sentences

for i in range(5):
  print(generator_2.make_sentence())


None
And so returne to you , he would be there to fetch him Bru .
None
Arme , Arme , and out , If this which he auouches , do 's condemne It selfe , for being there Cath .
None


In [ ]:
# Print 100 characters or less sentences

for i in range(5):
  print(generator_2.make_short_sentence(max_chars=100))

Heere comes the Generall , Brutus is tane my Lord Ant .
Oh Gertrude , come away , & c.
God has giuen you one pace , and you call him a - sleepe , leaues him .
Alas you know not , Sir Brut .
I did send To you for Gold to pay my Legions , Which you thought had been our innocent selfe .
